In [15]:
import os
import nltk
from pprint import pprint
import xml.etree.ElementTree as ET
from nltk.tag import StanfordPOSTagger
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from gensim.models import Word2Vec
from collections import defaultdict
import operator

/root/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [16]:
word2vecmodel = Word2Vec.load_word2vec_format('/home/word2vec/GoogleNews-vectors-negative300.bin', binary=True)
st = StanfordPOSTagger('/root/Downloads/stanford-postagger-full-2016-10-31/models/english-bidirectional-distsim.tagger',
                       '/root/Downloads/stanford-postagger-full-2016-10-31/stanford-postagger.jar')

st_words = stopwords.words('english')
st_words.append('anymore')
st_words.append('become')
st_words.append('yet')
st_words.append('ago')
st_words.append('earlier')
st_words.append('back')
st_words.append('soon')

In [34]:
tree = ET.parse('/home/PycharmProjects/Projects/pun_detection/data/trial/subtask3-homographic-test.xml')
root = tree.getroot()

In [35]:
word_freq = defaultdict()
for i, sent in enumerate(root):
    for word in sent:
        if (word.text not in word_freq):
            word_freq[word.text] = 0
        word_freq[word.text] = word_freq.get(word.text) + 1
sorted_word_freq = sorted(word_freq.items(), key=operator.itemgetter(1), reverse=True)
for x in sorted_word_freq:
    if (x[1] >= 25):
        st_words.append(x[0].lower())

In [36]:
def word2vec_sim(sent1,sent2):
    sent1_fil = [s for s in sent1 if s in word2vecmodel.vocab]
    sent2_fil = [s for s in sent2 if s in word2vecmodel.vocab]
    if(len(sent1_fil)>0 and len(sent2_fil)>0):
        return word2vecmodel.n_similarity(sent1_fil,sent2_fil)
    return 0.

In [37]:
puns = []
for child in root:
    words = " ".join(w.text for w in child)
    for w in child:
        if w.attrib['senses'] == '2':
            pun_word = w.text
    puns.append((words, pun_word))
for i, pun in enumerate(puns):
    print(i, pun)

0 ("Wal - Mart isn ' t the only saving place !", 'saving')
1 ('Can honeybee abuse lead to a sting operation ?', 'sting')
2 ('A ditch digger was entrenched in his career .', 'entrenched')
3 ("She was only a Blacksmith ' s daughter , but she knew how to forge ahead .", 'forge')
4 ("Did you hear about the new pinata ? It ' s a huge hit .", 'hit')
5 ('A bank manager who was also a high jumper spent most of his time in the vault .', 'vault')
6 ('A discussion of digging a new mine shaft was too deep for him .', 'deep')
7 ("She was suspected of stealing a brooch but they couldn ' t pin it on her .", 'pin')
8 ('Once ice cream was invented the problem was licked .', 'licked')
9 ("' ' There ' s room for one more , ' ' Tom admitted .", 'admitted')
10 ('They threw a party for the inventor of the toaster . And he was toasted .', 'toasted')
11 ("If you ' re a gardener you might call yourself a ' plant manager ' .", 'plant')
12 ('My advanced geometry class is full of squares .', 'squares')
13 ('Jesus

In [45]:
def find_pun_senses(words, anchor):
    line_output = defaultdict()
#     print(words)
    anchor_index = words.index(anchor[0])
#     print(anchor_index + 1)
    text_pos = st.tag(words)
    # print(text_pos)
    try:
        if (text_pos[anchor_index][1][0] == 'N'
            or text_pos[anchor_index][1][0] == 'V'
            or text_pos[anchor_index][1][0] == 'R'
            or text_pos[anchor_index][1][0] == 'J'):

            anchor_synsets = wn.synsets(anchor[0])
            # anchor_synsets.remove(original)
            # print(anchor_synsets)
            words = [t[0] for t in text_pos if t[0]!=anchor]
            
            for asyn in anchor_synsets:
                syn_damp = 0.5
                if(asyn.pos()==text_pos[anchor_index][1][0].lower()):
                    syn_damp = 1.0 
                score = 0.
                asyn_words = [w for w in asyn.definition().split(' ') if
                              w.lower() not in st_words]
                
                sent_sim = word2vec_sim(asyn_words, words)
                
                syn_score = 0.
                syn_sent_score = 0.
                                
                for t in text_pos:
                    if (t[0].lower() not in st_words and t[0] != anchor):
                        damp = 0.5
                        if (t[1][0].lower() == asyn.pos()
                            or (t[1][0].lower() == 'j' and asyn.pos() == 's')
                            or (t[1][0].lower() == 'j' and asyn.pos() == 'a')):
                            damp = 1.0
                    
                        if (len(asyn_words) > 0 and len(t) > 0):
                            sim = word2vec_sim(asyn_words, [t[0]])
                            score += (sim * damp)
                            # print((t[0], asyn.definition(), sim, sim * damp))
                    
                        t_synsets = wn.synsets(t[0])

                        for ts in t_synsets:
                            tsw = [w for w in ts.definition().split(' ') if
                                   w.lower() not in st_words]
                            syn_sim = word2vec_sim(asyn_words, tsw)
                            word_fil = [w for w in words if w!=t[0]]
                            syn_sent_sim = word2vec_sim(word_fil, tsw)
                                
                            if (syn_sim > syn_score):
                                syn_score = syn_sim
                            if (syn_sent_sim > syn_sent_score):
                                syn_sent_score = syn_sent_sim
                
                score = score* syn_damp    
#                 print((asyn.definition(), score, syn_score,syn_sent_score))
                line_output[str(asyn.lemmas()[0].key()) + '\t' +
                            str(asyn.definition())+'\t'+str(asyn.pos())] = score + syn_score*0.5 + syn_sent_score
    except:
        raise
    return line_output

def print_pun_senses(text,pun_word):
    output = find_pun_senses(text,[pun_word])
    sorted_out = sorted(output.items(), key=operator.itemgetter(1), reverse=True)
    for o in sorted_out:
        key_token = o[0].split('\t')
        print(str(key_token) + '\t' + str(o[1]) + '\n')

In [46]:
def run():
    with open('log_update_3.txt', 'w') as fw:
        with open('output_3.txt', 'w') as fo:
            for i, sent in enumerate(root):
                words = [word.text for word in sent]
                anchor = [word.text for word in sent if word.attrib['senses'] == '2']
                anchor_id = [word.attrib['id'] for word in sent if word.attrib['senses'] == '2']
                output = find_pun_senses(words, anchor)
                print(i)
                
                fw.write(str(words) + '\n')
                sorted_out = sorted(output.items(), key=operator.itemgetter(1), reverse=True)
                result = []
                result.append(anchor_id[0])
                for o in sorted_out:
                    key_token = o[0].split('\t')
                    result.append(key_token[0])
                    fw.write(str(key_token[1]) + '\t' + str(o[1]) + '\n')
                fw.write(' '.join(result) + '\n')
                fo.write(' '.join(result[:3]))
                fo.write('\n')
                fw.write('\n')


In [47]:
print_pun_senses(puns[8][0].split(),puns[8][1])

['cream%2:35:03::', 'beat thoroughly and conclusively in a competition or fight', 'v']	1.75728272845

['solve%2:31:00::', 'find the solution to (a problem or question) or understand the meaning of', 'v']	1.65362749851

['lap%2:34:00::', 'take up with the tongue', 'v']	1.55650500279

['lick%2:35:00::', 'pass the tongue over', 'v']	1.46715682851

['licked%5:00:00:defeated:00', 'having been got the better of', 's']	1.09044768868

